# Visualize light curves for all the sources of a region and skymaps of its fields 

In [3]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux

#Setup region and plotting paramters
region_name = "TDS_ELAISN1"

lc_block_srcs = 5 #Setup light curve plotting parameters
show_lc_pars = ["rg_src_id","nr_det","flux","flux_cpval","flux_nxv","flux_rchiq"]
show_map_pars = ["rg_src_id","ra","dec","pos_err","pos_nxv","nr_det","flux","flux_cpval", "nr_fd_srcs"]
sort_par = "rg_src_id" # "flux" #"rg_src_id" #

#Load region
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)
tt_srcs = rg.tt_sources[rg.tt_sources["sel"]]
nr_srcs = len(tt_srcs)
tt_srcs.sort(sort_par)
#display(tt_srcs)

### Draw light curves

In [7]:
nr_blocks = int(nr_srcs/lc_block_srcs)+1 
fig_lc = plt.figure("Light curves",figsize=(10, 9), constrained_layout=True)
flux_ylim = mag2flux([24.5,14.5])/1e-6
def plot_lc_range(block_nr):
    plt.cla()
    lc_min = lc_block_srcs * block_nr
    lc_max = lc_min + lc_block_srcs if lc_min + lc_block_srcs<=nr_srcs-1 else nr_srcs-1
    vvis.plot_light_curve(rg,fig=fig_lc, rg_src_ids = tt_srcs["rg_src_id"][lc_min:lc_max],ylim = flux_ylim.value ) #, 
    plt.show()
    display(tt_srcs[lc_min:lc_max][show_lc_pars])
widgets.interactive( plot_lc_range, block_nr=(0,nr_blocks-1))

interactive(children=(IntSlider(value=14, description='block_nr', max=28), Output()), _dom_classes=('widget-in…

### Draw sky maps of fields

In [3]:
fig_skymap = plt.figure("Sky map",figsize=(8, 7), constrained_layout=True)
rg.tt_sources.add_index("rg_src_id")
def plot_sky_field(rg_fd_id):
    plt.cla()
    fd = rg.get_field(rg_fd_id=rg_fd_id, load_method="FITS", add_field=True)
    fig, ax = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = fd.get_sky_region(),fig=fig_skymap) # 
    ax, rg_src_ids = vvis.plot_sky_sources(rg.tt_sources,tt_det=None, sky_region_wcs = (fd.get_sky_region(),fd.ref_wcs)) # 
    src_idx = rg.tt_sources.loc_indices["rg_src_id", rg_src_ids]
    plt.show()
    display(rg.tt_sources[src_idx][show_map_pars])
widgets.interactive( plot_sky_field, rg_fd_id=(rg.tt_fields["rg_fd_id"].min(),rg.tt_fields["rg_fd_id"].max()))

interactive(children=(IntSlider(value=0, description='rg_fd_id', max=1), Output()), _dom_classes=('widget-inte…